In [48]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline
#https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.


DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import logging 
import joblib
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import algo_ecg.data
import algo_ecg.preprocessing
#import algo_ecg.model
from algo_ecg.feature_transformer import \
    AllFeatureCustomTransformer \

import sklearn.model_selection
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer, StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

import statsmodels.api as sm 
from scipy.stats import norm


In [50]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.getLogger('matplotlib.font_manager').disabled = True
plt.rcParams['figure.figsize'] = [12, 8]

In [15]:
#X_in, y, pids = algo_ecg.data.import_physionet_data('../data', num_files_to_read=1000)
X_in, y, pids = algo_ecg.data.import_physionet_data('/Users/jessie/data/PhysioNet_CinC', num_files_to_read=1000)
#X_in, y = algo_ecg.data.import_physionet_data('../data')

In [16]:
X, pids = algo_ecg.preprocessing.preprocess_pid(X_in, pids)

DEBUG:root:
DEBUG:root:i=0, pid=A00001: 37 outliers found
DEBUG:root:
DEBUG:root:i=1, pid=A00002: 48 outliers found
DEBUG:root:
DEBUG:root:i=2, pid=A00003: 6 outliers found
DEBUG:root:i=2, pid=A00003: flipped values
DEBUG:root:
DEBUG:root:i=3, pid=A00003: 45 outliers found
DEBUG:root:i=3, pid=A00003: flipped values
DEBUG:root:
DEBUG:root:i=4, pid=A00004: 99 outliers found
DEBUG:root:
DEBUG:root:i=5, pid=A00005: 18 outliers found
DEBUG:root:
DEBUG:root:i=6, pid=A00005: 44 outliers found
DEBUG:root:
DEBUG:root:i=7, pid=A00006: 28 outliers found
DEBUG:root:
DEBUG:root:i=8, pid=A00007: 53 outliers found
DEBUG:root:i=8, pid=A00007: flipped values
DEBUG:root:
DEBUG:root:i=9, pid=A00009: 19 outliers found
DEBUG:root:
DEBUG:root:i=10, pid=A00010: 39 outliers found
DEBUG:root:
DEBUG:root:i=11, pid=A00010: 119 outliers found
DEBUG:root:
DEBUG:root:i=13, pid=A00012: 101 outliers found
DEBUG:root:
DEBUG:root:i=14, pid=A00016: 92 outliers found
DEBUG:root:i=14, pid=A00016: flipped values
DEBUG:root

In [17]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=.7)

In [18]:
slice_length = len(X_train[0])

X_train = np.concatenate(X_train).reshape(-1, slice_length, 1)
X_test = np.concatenate(X_test).reshape(-1, slice_length, 1)

In [19]:
X_train_pd = pd.DataFrame.from_records(X_train)
X_test_pd = pd.DataFrame.from_records(X_test)
                 

In [20]:
pipe = Pipeline(steps=[
#    ('peaks', FindPeaksCustomTransformer (axis=1)), 
#    ('feature_select_manual',FunctionTransformer(select_cols, kw_args={"col_names": ['rmssd']})), 
    ('features_all',AllFeatureCustomTransformer(axis = 1)), 
    ('feature_selection', SelectFromModel(LinearSVC(dual="auto", penalty="l1", C = 0.01))), 
    ('standardscaler', StandardScaler()) 
#    ("lr", LogisticRegression(max_iter=1000, tol=0.1, class_weight = "balanced"))
     ]
    )


In [21]:
transformed_np_array = pipe.fit_transform(X_train_pd, y_train)

/Users/jessie/.pyenv/versions/cinc/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [22]:
pipe.named_steps.feature_selection.get_feature_names_out()

array(['Mean_RR_ms', 'STD_RR_ms', 'Mean_HR_beats_per_min_kubio',
       'Mean_HR_beats_per_min', 'Max_HR_beats_per_min', 'RMSSD_ms',
       'pNN50', 'pNN70', 'Power_VLF_ms2', 'Power_LF_ms2', 'Power_HF_ms2',
       'Power_Total_ms2'], dtype=object)

In [23]:
transformed_df = pd.DataFrame(transformed_np_array, 
                              columns=pipe.named_steps.feature_selection.get_feature_names_out())

In [28]:
X = sm.add_constant(transformed_df) 
log_reg = sm.Logit(y_train, X)
result = log_reg.fit() 

Optimization terminated successfully.
         Current function value: 0.132854
         Iterations 9


In [30]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  482
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                           11
Date:                Mon, 04 Mar 2024   Pseudo R-squ.:                  0.6735
Time:                        12:09:47   Log-Likelihood:                -64.035
converged:                       True   LL-Null:                       -196.13
Covariance Type:            nonrobust   LLR p-value:                 2.954e-50
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -4.2251      0.527     -8.023      0.000      -5.257      -3.193
Mean_RR_ms                     -5.2060      2.359     -2.207      0.027      -9.829      -0.583
STD_RR_ms                       0.5542      2.286      0.242      0.808      -3.926       5.034
Mean_HR_beats_per_min_kubio     5.5208      9.389      0.588      0.557     -12.881      23.923
Mean_HR_beats_per_min         -11.6238      9.457     -1.229      0.219     -30.159       6.912
Max_HR_beats_per_min           -0.9696      0.636     -1.526      0.127      -2.215       0.276
RMSSD_ms                        2.1564      1.061      2.032      0.042       0.076       4.237
pNN50                           1.8538      0.678      2.733      0.006       0.524       3.183
pNN70                           1.0138      0.686      1.478      0.139      -0.330       2.358
Power_VLF_ms2                  -0.5101   3.95e+06  -1.29e-07      1.000   -7.73e+06    7.73e+06
Power_LF_ms2                    0.1590   6.56e+06   2.42e-08      1.000   -1.29e+07    1.29e+07
Power_HF_ms2                   -0.2529    1.4e+07  -1.81e-08      1.000   -2.75e+07    2.75e+07
Power_Total_ms2                -0.2114   2.13e+07   -9.9e-09      1.000   -4.18e+07    4.18e+07
===============================================================================================
"""

Summary: 
- The coefficients are log odds that can be estimated using maximum likelihood estimation.
- Once the parameter estimates are obtained, the standard errors can be calculated using formulas derived from the Fisher Information Matrix or through numerical approximations.
- The z-score is computed as the ratio of the coefficient estimate to its standard error.
- P value is calculated using Wald test, which utilizes the z-score.
- Once you have the Wald statistic, you can then calculate the p-value associated with the coefficient using the standard normal distribution (under the assumption of large sample sizes). The p-value represents the probability of observing a Wald statistic as extreme as the one computed, assuming the null hypothesis is true.

#### The general steps to calculate standard errors for logistic regression coefficients are as follows:

Compute the Hessian Matrix (Information Matrix):
- Calculate the Hessian matrix, which is the matrix of second partial derivatives of the log-likelihood function with respect to the parameters. For logistic regression, this involves taking the second derivatives of the log-likelihood function.
    
Compute the Variance-Covariance Matrix:
- Take the negative inverse of the Hessian matrix to obtain the variance-covariance matrix. The variance-covariance matrix provides the variances of the parameter estimates on the diagonal and the covariances between parameter estimates off the diagonal. 

Extract Standard Errors:
- Extract the square root of the diagonal elements of the variance-covariance matrix to obtain the standard errors for each parameter.

In [35]:
# Get the covariance matrix of the parameters
cov_matrix = result.cov_params()

# Extract standard errors from the diagonal elements
se = np.sqrt(np.diag(cov_matrix))


In [36]:
cov_matrix

,const,Mean_RR_ms,STD_RR_ms,Mean_HR_beats_per_min_kubio,Mean_HR_beats_per_min,Max_HR_beats_per_min,RMSSD_ms,pNN50,pNN70,Power_VLF_ms2,Power_LF_ms2,Power_HF_ms2,Power_Total_ms2
const,0.277324,0.210315,-0.430947,-1.797776,2.094533,0.090608,-0.081420,-0.133057,0.022179,-1.336351e-01,-3.295668e-01,-6.797622e-01,1.020531e+00
Mean_RR_ms,0.210315,5.564614,-0.722100,4.838861,0.458639,0.186173,0.357057,-0.219713,0.182755,-9.784523e-02,-1.646664e-01,-1.016636e+00,1.135959e+00
STD_RR_ms,-0.430947,-0.722100,5.224106,16.996146,-18.056333,-0.843885,-0.971757,-0.029156,-0.144887,1.233889e+00,2.502443e+00,5.630049e+00,-7.922709e+00
Mean_HR_beats_per_min_kubio,-1.797776,4.838861,16.996146,88.152237,-86.166781,-2.119328,0.573773,-0.150942,-0.973944,4.516083e+00,9.032166e+00,1.970654e+01,-2.627539e+01
Mean_HR_beats_per_min,2.094533,0.458639,-18.056333,-86.166781,89.435327,2.292823,-0.393396,-0.087679,1.171722,-4.722761e+00,-9.412481e+00,-2.116116e+01,2.797380e+01
Max_HR_beats_per_min,0.090608,0.186173,-0.843885,-2.119328,2.292823,0.403893,0.078592,0.035436,-0.048869,-1.305771e-01,-3.695053e-01,-8.223577e-01,1.233982e+00
RMSSD_ms,-0.081420,0.357057,-0.971757,0.573773,-0.393396,0.078592,1.126597,0.019259,-0.144818,-2.551141e-01,-3.815446e-01,-9.521580e-01,1.374908e+00
pNN50,-0.133057,-0.219713,-0.029156,-0.150942,-0.087679,0.035436,0.019259,0.460013,-0.355769,1.853183e-01,2.678880e-01,5.885989e-01,-8.840478e-01
pNN70,0.022179,0.182755,-0.144887,-0.973944,1.171722,-0.048869,-0.144818,-0.355769,0.470284,-1.871464e-01,-2.902679e-01,-6.569221e-01,9.319128e-01
Power_VLF_ms2,-0.273775,-0.115706,3.371589,12.229389,-12.615952,-0.463525,-0.664046,0.140160,-0.213282,1.556369e+13,2.587045e+13,5.526185e+13,-8.420732e+13


In [37]:
se

array([5.26616005e-01, 2.35894345e+00, 2.28563024e+00, 9.38894228e+00,
       9.45702528e+00, 6.35525857e-01, 1.06141253e+00, 6.78242538e-01,
       6.85772206e-01, 3.94508437e+06, 6.55764300e+06, 1.40077745e+07,
       2.13448728e+07])

In [40]:
coefficients

const                          -4.225064
Mean_RR_ms                     -5.206043
STD_RR_ms                       0.554184
Mean_HR_beats_per_min_kubio     5.520755
Mean_HR_beats_per_min         -11.623765
Max_HR_beats_per_min           -0.969563
RMSSD_ms                        2.156424
pNN50                           1.853806
pNN70                           1.013823
Power_VLF_ms2                  -0.510124
Power_LF_ms2                    0.159010
Power_HF_ms2                   -0.252931
Power_Total_ms2                -0.211419
dtype: float64

In [44]:
# Calculate z-scores

coefficients = result.params

z_scores = coefficients / se

# Calculate two-tailed p-values
p_values = 2 * (1 - norm.cdf(np.abs(z_scores)))

In [45]:
p_values

array([1.11022302e-15, 2.73183547e-02, 8.08420164e-01, 5.56528195e-01,
       2.19028942e-01, 1.27107685e-01, 4.21886376e-02, 6.27129368e-03,
       1.39309728e-01, 9.99999897e-01, 9.99999981e-01, 9.99999986e-01,
       9.99999992e-01])

In [47]:
# Display results
results_df = pd.DataFrame({'Coefficient': coefficients, 'Standard Error': se, 'Z-Score': z_scores, 'P-Value': p_values}, index=X.columns)
print(results_df)

                             Coefficient  Standard Error       Z-Score  \
const                          -4.225064    5.266160e-01 -8.023046e+00   
Mean_RR_ms                     -5.206043    2.358943e+00 -2.206939e+00   
STD_RR_ms                       0.554184    2.285630e+00  2.424646e-01   
Mean_HR_beats_per_min_kubio     5.520755    9.388942e+00  5.880061e-01   
Mean_HR_beats_per_min         -11.623765    9.457025e+00 -1.229114e+00   
Max_HR_beats_per_min           -0.969563    6.355259e-01 -1.525607e+00   
RMSSD_ms                        2.156424    1.061413e+00  2.031655e+00   
pNN50                           1.853806    6.782425e-01  2.733249e+00   
pNN70                           1.013823    6.857722e-01  1.478366e+00   
Power_VLF_ms2                  -0.510124    3.945084e+06 -1.293062e-07   
Power_LF_ms2                    0.159010    6.557643e+06  2.424811e-08   
Power_HF_ms2                   -0.252931    1.400777e+07 -1.805645e-08   
Power_Total_ms2                -0.2114